Predicted Values

In [ ]:
# Requirements
%pip install pandas numpy matplotlib fredapi python-dotenv scipy scikit-learn joblib tdqm xgboost

In [ ]:
from src import *

2017-02-08 start test to 2018-02-07

2014-03-03 start CV to 2016-10-03

In [ ]:
with open('src/processed_data.pkl', 'rb') as f:
    op, high, low, close, volume, returns, risk_free_rate = pickle.load(f)

train_returns = returns.iloc[:-252]
test_returns = returns.iloc[-252:]
test_risk_free_rate = risk_free_rate.iloc[-252:]
cv_returns = train_returns.iloc[-252:]
cv_returns

Predicting the returns

In [ ]:
predictor = MLReturnPredictor(model_type='ridge', alpha=.1)
X_train, y_train = predictor.create_training_dataset(os.listdir('src/data')[:os.listdir('src/data').index('2017-02-08.pkl')-65],
                                                      train_returns.iloc[:-65]) #65 for purge
predictor.train(X_train, y_train)
predicted = predictor.predict_all_test_returns(os.listdir('src/data')[os.listdir('src/data').index('2017-02-08.pkl'):])

This is during the test period then to do walk forward can use this code

In [92]:
from sklearn.preprocessing import StandardScaler
with open('src/data_weekly_ret/return_5_targets.pkl', 'rb') as f:
    weekly_volatility_targets = pickle.load(f)

mask = (weekly_volatility_targets.index >= '2015-10-05') & (weekly_volatility_targets.index <= '2016-10-03')
testing_data = weekly_volatility_targets.loc[mask]
testing_dates = testing_data.index.strftime('%Y-%m-%d').map(lambda x: x+".pkl").values

preds = []

for test_date in testing_dates[:5]:
    predictor = MLReturnPredictor(model_type='ridge', alpha=11)
    v = os.listdir('src/data_weekly_ret')[:os.listdir('src/data_weekly_ret').index(test_date)-20]
    print(v)
    print(test_date)
    X_train, y_train = predictor.create_training_dataset(v,weekly_volatility_targets)
    X_train = StandardScaler().fit_transform(X_train)
    predictor.train(X_train, y_train)
    predicted = predictor.predict_all_test_returns([test_date])
    preds.append(predicted)

predicted = pd.concat(preds)
print(predicted)

['2013-05-28.pkl', '2013-06-03.pkl', '2013-06-10.pkl', '2013-06-17.pkl', '2013-06-24.pkl', '2013-07-01.pkl', '2013-07-08.pkl', '2013-07-15.pkl', '2013-07-22.pkl', '2013-07-29.pkl', '2013-08-05.pkl', '2013-08-12.pkl', '2013-08-19.pkl', '2013-08-26.pkl', '2013-09-03.pkl', '2013-09-09.pkl', '2013-09-16.pkl', '2013-09-23.pkl', '2013-09-30.pkl', '2013-10-07.pkl', '2013-10-14.pkl', '2013-10-21.pkl', '2013-10-28.pkl', '2013-11-04.pkl', '2013-11-11.pkl', '2013-11-18.pkl', '2013-11-25.pkl', '2013-12-02.pkl', '2013-12-09.pkl', '2013-12-16.pkl', '2013-12-23.pkl', '2013-12-30.pkl', '2014-01-02.pkl', '2014-01-06.pkl', '2014-01-13.pkl', '2014-01-21.pkl', '2014-01-27.pkl', '2014-02-03.pkl', '2014-02-10.pkl', '2014-02-18.pkl', '2014-02-24.pkl', '2014-03-03.pkl', '2014-03-10.pkl', '2014-03-17.pkl', '2014-03-24.pkl', '2014-03-31.pkl', '2014-04-07.pkl', '2014-04-14.pkl', '2014-04-21.pkl', '2014-04-28.pkl', '2014-05-05.pkl', '2014-05-12.pkl', '2014-05-19.pkl', '2014-05-27.pkl', '2014-06-02.pkl', '2014-06-

In [83]:
r2_score(testing_data.loc[predicted.index], predicted)

-32.29124454909616

In [90]:
with open('src/data_weekly_ret/return_5_targets.pkl', 'rb') as f:
    weekly_volatility_targets = pickle.load(f)

In [88]:
for i in range(X_train.T.shape[0]):
    print(f"Feature {i} correlation with target: {np.corrcoef(X_train.T[i], y_train)[0, 1]}")


Feature 0 correlation with target: 0.00557653747487012
Feature 1 correlation with target: -0.002889947742665886
Feature 2 correlation with target: 0.006083572280014524
Feature 3 correlation with target: -0.017528471033751724
Feature 4 correlation with target: 0.006078738894112105
Feature 5 correlation with target: -0.01757119806069574
Feature 6 correlation with target: 0.027541770794512206
Feature 7 correlation with target: 0.013548175505385866
Feature 8 correlation with target: 0.0018885722646624107
Feature 9 correlation with target: -0.0056601455956081775
Feature 10 correlation with target: -0.032253608039023975
Feature 11 correlation with target: -0.03214860580980798
Feature 12 correlation with target: -0.00660121221558977
Feature 13 correlation with target: -0.002167680821447501
Feature 14 correlation with target: -0.028697635360370767
Feature 15 correlation with target: -0.01944635227192168
Feature 16 correlation with target: -0.01279296752484237
Feature 17 correlation with target

Purge period either 70 trading days, 14 trasing weeks, 4 trading months

In [ ]:
# testing_dates = test_returns.index.strftime('%Y-%m-%d').map(lambda x: x+".pkl").values

with open('src/data_weekly_var/weekly_volatility_targets.pkl', 'rb') as f:
    weekly_volatility_targets = pickle.load(f)

test_weekly_var = weekly_volatility_targets.loc['2017-02-08':]

testing_dates = test_weekly_var.index.strftime('%Y-%m-%d').map(lambda x: x+".pkl").values

preds = []

for test_date in testing_dates:
    predictor = MLReturnPredictor(model_type='ridge', alpha=.1)
    X_train, y_train = predictor.create_training_dataset(os.listdir('src/data_weekly_var')[:os.listdir('src/data_weekly_var').index(test_date)-10],
                                                        np.log(weekly_volatility_targets)) #65 for purge
    predictor.train(X_train, y_train)
    predicted = predictor.predict_all_test_returns([test_date])
    preds.append(predicted)

predicted = pd.concat(preds)
predicted = np.exp(predicted)


Generate weights for variance

In [ ]:
tickers = predicted.columns.tolist()

def compute_weights_for_date(date):
    opt = PortfolioOptimizer(returns.loc[:date].iloc[:-1]).minimum_variance_from_cov(predicted.loc[[date]], max_position=0.05)
    return pd.Series(opt, index=tickers)

weights_list = Parallel(n_jobs=-1)(
    delayed(compute_weights_for_date)(d) for d in tqdm(predicted.index, desc="weights")
)

weights = pd.concat(weights_list, axis=1).T
weights.index.name = 'date'
weights.index = predicted.index

with open('weights_variance.pkl', 'wb') as f:
    pickle.dump(weights, f)

The one below takes very long btw

Chooses optimal portfolio based on predicted returns

In [ ]:
tickers = predicted.columns.tolist()

def compute_weights_for_date(date):
    predicted_returns = pd.concat([returns.loc[:date].iloc[:-1], predicted.loc[[date]]])
    opt = PortfolioOptimizer(predicted_returns).minimum_variance(max_position=0.05)
    return pd.Series(opt, index=tickers)

weights_list = Parallel(n_jobs=-1)(
    delayed(compute_weights_for_date)(d) for d in tqdm(predicted.index[:5], desc="weights")
)

weights = pd.concat(weights_list, axis=1).T
weights.index.name = 'date'
weights.index = predicted.index[:5]

with open('weights.pkl', 'wb') as f:
    pickle.dump(weights, f)


In [ ]:
returns.head(5).sum(axis=0)

In [ ]:
date_idx = returns.index.get_loc("2018-02-07")
window_start = date_idx - 70
window_end = date_idx
returns_window = returns.iloc[window_start:window_end]
returns_window

In [ ]:
spy_weights = volume.shift(1)
close_prices = close.shift(1)
spy_weights = spy_weights.loc["2017-02-08":"2018-02-07"]
close_prices = close_prices.loc["2017-02-08":"2018-02-07"]
spy_weights = spy_weights*close_prices
spy_weights = spy_weights.div(spy_weights.sum(axis=1), axis=0)
spy_weights.head()

Preparing backtest


In [ ]:
backtester = WalkForwardBacktest(
    train_returns=train_returns,
    test_returns=test_returns,
    # test_prices=test_prices,
    rf_rate_test=test_risk_free_rate,
    rebalance_freq='D'  # Weekly
)

running backtest

In [ ]:
results = {}

with open('weights.pkl', 'rb') as f:
    weights = pickle.load(f)

# Run Equal Weight
results['ML w fees'] = backtester.run_backtest_preloaded_weights(
    strategy_name='ML',
    preloaded_weights=weights,
    transaction_cost_bps=10
)

results['ML no fees'] = backtester.run_backtest_preloaded_weights(
    strategy_name='ML',
    preloaded_weights=weights,
    transaction_cost_bps=0
)


weights = weights.iloc[[0]]

for col in weights.columns:
    weights[col].values[:] = 1/470

results['Base - Buy & Hold'] = backtester.run_backtest_preloaded_weights(
    strategy_name='ML',
    preloaded_weights=weights,
    transaction_cost_bps=0
)

with open('weights_variance.pkl', 'rb') as f:
    weights_var = pickle.load(f)

results['vol'] = backtester.run_backtest_preloaded_weights(
    strategy_name='Volatility',
    preloaded_weights=weights_var,
    transaction_cost_bps=0
)

Evaluation

In [ ]:
results['vol']

In [ ]:
evaluator = PerformanceEvaluator(test_risk_free_rate)

comparison_df = evaluator.compare_strategies(results)
print("\nPerformance Comparison:")
print(comparison_df.to_string())

# ============================================================
# STEP 5: Visualize Results
# ============================================================
print("\n5. Generating Visualizations")
print("-" * 60)

fig = evaluator.plot_results(results, figsize=(15, 10))
plt.show()

fred = Fred(api_key = os.getenv("API_KEY"))
spy = fred.get_series_latest_release('SP500').loc["2017-02-08":"2018-02-07"]
spy.plot()
plt.show()
